In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob
import urllib.parse
import time
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
api_key = os.getenv('News_API_KEY')
if not api_key:
    raise ValueError("API key is missing. Set the News_API_KEY environment variable in the .env file.")
tickers = ["Apple", "Google", "Microsoft", "Amazon", "Meta", "Nvidia", "Tesla", "Netflix", "AMD", "Palanir"]
base_url = f"https://newsapi.org/v2/everything?"
results = []

In [4]:
for i in range(1):
    
    date = datetime.utcnow() - timedelta(days = i)
    from_date = to_date = date.strftime("%Y-%m-%d")

    for ticker in tickers:
        print(f"Pulling news for {ticker} on {from_date}")

        params = {
            "q" : f'"{ticker}" AND stock',
            "searchIn": "title,description",
            "from": from_date,
            "to": to_date,
            "language": "en",
            "sortBy": "relevancy",
            "pageSize": 10,
            "apiKey": api_key
        }
        try:
            response = requests.get(base_url, params = params)
            response.raise_for_status()
            articles = response.json().get('articles', [])
        except Exception as e:
            print(f"Error fetching data for {ticker} on {from_date}: {e}")
            continue
        
        sentiments = []
        for article in articles:
            title = article.get('title', '')
            description = article.get('description', '')
            text = f"{title} {description}".strip()
            if text:
                score = TextBlob(text).sentiment.polarity
                sentiments.append(score)

        avg_sentiment = sum(sentiments) / len(sentiments) if sentiments else None
        
        results.append({
            "ticker": ticker,
            "date": from_date,
            "article_count": len(sentiments),
            "avg_sentiment":avg_sentiment
        })
        time.sleep(1.1)

/var/folders/54/hwsm49bs033bpdy18h8fnp2h0000gn/T/ipykernel_75523/4215040265.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date = datetime.utcnow() - timedelta(days = i)


Pulling news for Apple on 2025-05-02
Error fetching data for Apple on 2025-05-02: 429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22Apple%22+AND+stock&searchIn=title%2Cdescription&from=2025-05-02&to=2025-05-02&language=en&sortBy=relevancy&pageSize=10&apiKey=ed97359fd5f145c393ca0e0f2683a396
Pulling news for Google on 2025-05-02
Error fetching data for Google on 2025-05-02: 429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22Google%22+AND+stock&searchIn=title%2Cdescription&from=2025-05-02&to=2025-05-02&language=en&sortBy=relevancy&pageSize=10&apiKey=ed97359fd5f145c393ca0e0f2683a396
Pulling news for Microsoft on 2025-05-02
Error fetching data for Microsoft on 2025-05-02: 429 Client Error:  for url: https://newsapi.org/v2/everything?q=%22Microsoft%22+AND+stock&searchIn=title%2Cdescription&from=2025-05-02&to=2025-05-02&language=en&sortBy=relevancy&pageSize=10&apiKey=ed97359fd5f145c393ca0e0f2683a396
Pulling news for Amazon on 2025-05-02
Error fetching data for

In [5]:
df = pd.DataFrame(results)
df.head()

""
